In [2]:
from paddleocr import PaddleOCR, draw_ocr

# Paddleocr目前支持的多语言语种可以通过修改lang参数进行切换
# 例如`ch`, `en`, `fr`, `german`, `korean`, `japan`
ocr = PaddleOCR(use_angle_cls=True, lang="en")  # need to run only once to download and load model into memory

ModuleNotFoundError: No module named 'paddleocr'

In [18]:
img_path = 's.jpg'
result = ocr.ocr(img_path, cls=True)
for idx in range(len(result)):
    res = result[idx]
    for line in res:
        print(line)

# for i in result[0]:
#     print(i)

from PIL import Image
result = result[0]
image = Image.open(img_path).convert('RGB')
image
# print(result)
# for line in result:
#     print(line[0])
# boxes = [line[0] for line in result]
txts = [line[1][0] for line in result]
txts
# scores = [line[1][1] for line in result]
# im_show = draw_ocr(image, boxes, txts, scores, font_path="./doc/fonts/simfang.ttf")
# im_show = Image.fromarray(im_show)
# im_show.save('result.jpg')

[2024/08/28 14:51:48] ppocr DEBUG: dt_boxes num : 10, elapsed : 0.14487552642822266
[2024/08/28 14:51:48] ppocr DEBUG: cls num  : 10, elapsed : 0.05011796951293945
[2024/08/28 14:51:48] ppocr DEBUG: rec_res num  : 10, elapsed : 0.6540014743804932
[[[343.0, 33.0], [621.0, 33.0], [621.0, 64.0], [343.0, 64.0]], ('Simple Flowchart', 0.9770476818084717)]
[[[63.0, 55.0], [114.0, 57.0], [113.0, 80.0], [62.0, 77.0]], ('Start', 0.9995299577713013)]
[[[618.0, 93.0], [692.0, 90.0], [694.0, 134.0], [619.0, 136.0]], ('QG', 0.9970448613166809)]
[[[586.0, 193.0], [627.0, 193.0], [627.0, 216.0], [586.0, 216.0]], ('End', 0.9994561076164246)]
[[[32.0, 224.0], [125.0, 224.0], [125.0, 245.0], [32.0, 245.0]], ('Process1', 0.9987485408782959)]
[[[310.0, 361.0], [383.0, 361.0], [383.0, 382.0], [310.0, 382.0]], ('Inspect', 0.9969881176948547)]
[[[31.0, 381.0], [123.0, 381.0], [123.0, 399.0], [31.0, 399.0]], ('Process2', 0.9979144334793091)]
[[[562.0, 373.0], [657.0, 376.0], [656.0, 397.0], [561.0, 394.0]], ('

['Simple Flowchart',
 'Start',
 'QG',
 'End',
 'Process1',
 'Inspect',
 'Process2',
 'Process3',
 'Accepted',
 'Rejected']

In [12]:
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [13]:
str= ' '.join(txts)
str


'START Read num reverse=0 tempNum=num No num!=0 Yes rem=num%10 reverse*=10+rem num=num/10 Yes reverse= tempNum No Palindrome not palindrome STOP'

In [14]:
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
# The following is the data containing the text from a flowchart image:
# The data is in the form of a list with the following elements : the 4 coordinates of the bounding box of the text, the text itself, and the confidence score of the text.
# Understand the data with the relation between the bounding box and the text.
# The data is as follows:
template = """
The following is the data containing the text from a flowchart image:
Explain the overall semantic meaning of the flowchart preferably in a structured format.
Answer using strictly the information provided in the text below and give the semantic meaning of the flowchart only
{txts}.
"""

formatted_template = template.format(txts=txts)
prompt = PromptTemplate.from_template(formatted_template)


In [15]:
llm = OpenAI(api_key=api_key,temperature=0.3)
llm_chain = prompt|llm


In [16]:
question = {"input": "Summarize the data from the flowchart image"}
print(llm_chain.invoke(question))



START:
- The flowchart begins at the START point.

Read num:
- The program reads a number from the user and assigns it to the variable 'num'.

reverse=0:
- The variable 'reverse' is initialized to 0.

tempNum=num:
- The value of 'num' is stored in the variable 'tempNum'.

No:
- This is a decision point where the program checks if the number is equal to 0.

num!=0:
- If the number is not equal to 0, the program proceeds to the next step.

Yes:
- This is a decision point where the program checks if the number is a palindrome.

rem=num%10:
- The remainder of the number divided by 10 is stored in the variable 'rem'.

reverse*=10+rem:
- The variable 'reverse' is multiplied by 10 and added to the value of 'rem'.

num=num/10:
- The number is divided by 10 and the result is stored in the variable 'num'.

Yes:
- If the number is a palindrome, the program proceeds to the next step.

reverse=:
- The value of 'reverse' is assigned to the variable 'tempNum'.

No:
- If the number is not a palindrom

In [17]:
#Add appropriate image preprocessing steps to improve the OCR results if needed.
#Limitations- horizontal text me problem in interpretation
#Limitations- text with arrows pe text me problem in interpretation
# End me just semantic meaning of flowchart ko match karana hai if similar then plaigiarsim hai